In [7]:
import requests
import json
import forecastio
from pprint import pprint
from geopy.geocoders import Nominatim
import pandas as pd
import import_ipynb
from Asheville1_airbnb import Asheville_airbnb
import datetime as dt

In [9]:
api_key = "a36c8345cc74c469a50078545e4970de"

,Date,Under $50,$50-100,$100-150,$150-200,$200-250
0,2018-12-21,75,370,234,120,63
1,2018-12-22,75,323,223,110,61
2,2018-12-23,97,408,196,93,58
3,2018-12-24,90,369,198,88,55
4,2018-12-25,92,361,197,88,57
5,2018-12-26,77,363,187,79,52
6,2018-12-27,77,347,153,58,47
7,2018-12-28,57,264,184,76,41


In [17]:
geolocator = Nominatim(user_agent="ETL_Project")

location = geolocator.geocode("Asheville, NC")

ASH_coord = []
ASH_coord.append(location.latitude)
ASH_coord.append(location.longitude)

ASH_coord

[35.6009498, -82.5540161]

In [18]:
forecast = forecastio.load_forecast(api_key, ASH_coord[0], ASH_coord[1])

byDay = forecast.daily()
print(byDay.summary)
print(byDay.icon)
print(byDay.data[4].temperatureHigh)

Mixed precipitation throughout the week, with high temperatures bottoming out at 42°F on Monday.
rain
53.42


In [19]:
temp_high = []
temp_low = []
precip_perc = []
precip_type = []
precip_int = []
date = []

for i in byDay.data:
    date.append(i.time.date())
    temp_high.append(i.temperatureHigh)
    temp_low.append(i.temperatureLow)
    precip_perc.append(i.precipProbability)
    precip_int.append(i.precipIntensity)
    try:
        precip_type.append(i.precipType)
    except:
        precip_type.append("none")

date

dates = [dt.datetime.strftime(i, '%Y-%m-%d') for i in date]
dates

['2018-12-22',
 '2018-12-23',
 '2018-12-24',
 '2018-12-25',
 '2018-12-26',
 '2018-12-27',
 '2018-12-28',
 '2018-12-29']

In [20]:
df = pd.DataFrame({"Date": dates, 
                     "High Temp": temp_high, 
                     "Low Temp": temp_low, 
                     "Precipitation Probability": precip_perc,
                     "Precipitation Intensity": precip_int,
                     "Precipitation Type": precip_type
                    })
df.dtypes

Date                          object
High Temp                    float64
Low Temp                     float64
Precipitation Probability    float64
Precipitation Intensity      float64
Precipitation Type            object
dtype: object

In [21]:
ASH_table = pd.merge(df, Asheville_airbnb, on="Date")
ASH_table

,Date,High Temp,Low Temp,Precipitation Probability,Precipitation Intensity,Precipitation Type,Under $50,$50-100,$100-150,$150-200,$200-250
0,2018-12-22,43.62,27.57,0.13,0.0010,snow,75,323,223,110,61
1,2018-12-23,52.73,26.53,0.25,0.0015,rain,97,408,196,93,58
2,2018-12-24,42.17,25.42,0.09,0.0003,snow,90,369,198,88,55
3,2018-12-25,51.52,26.17,0.00,0.0000,none,92,361,197,88,57
4,2018-12-26,53.42,35.53,0.00,0.0000,none,77,363,187,79,52
5,2018-12-27,49.98,51.27,0.91,0.0383,rain,77,347,153,58,47
6,2018-12-28,65.88,39.36,0.88,0.0402,rain,57,264,184,76,41
